# Подготовка

In [1]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .getOrCreate()

Датасеты можно скачать из репозитория 

https://github.com/databricks/LearningSparkV2/tree/master/databricks-datasets/learning-spark-v2/flights

https://github.com/databricks/LearningSparkV2/tree/master/databricks-datasets/learning-spark-v2/sf-fire

In [2]:
airport_codes = spark.read \
    .option("header", "true").option("delimiter", "\t").option("inferSchema", "true") \
    .csv("../datasets/airport-codes-na.txt") \
    .persist(pyspark.StorageLevel.DISK_ONLY)
airport_delays = spark.read.option("header", "true").option("inferSchema", "true") \
    .csv("../datasets/departuredelays.csv") \
    .persist(pyspark.StorageLevel.DISK_ONLY)

fire_calls = spark.read.option("header", "true").option("inferSchema", "true") \
    .csv("../datasets/sf-fire-calls.csv") \
    .persist(pyspark.StorageLevel.DISK_ONLY)
fire_incidents = spark.read.option("header", "true").option("inferSchema", "true") \
    .csv("../datasets/sf-fire-incidents.csv") \
    .persist(pyspark.StorageLevel.DISK_ONLY)

# Пример работы оптимизатора

In [5]:
airport_delays \
    .join(airport_codes, airport_codes['IATA'] == airport_delays['origin']) \
    .filter(col('State') == 'LA') \
    .explain(True)

== Parsed Logical Plan ==
'Filter ('State = LA)
+- Join Inner, (IATA#20 = origin#65)
   :- Relation [date#62,delay#63,distance#64,origin#65,destination#66] csv
   +- Relation [City#17,State#18,Country#19,IATA#20] csv

== Analyzed Logical Plan ==
date: int, delay: int, distance: int, origin: string, destination: string, City: string, State: string, Country: string, IATA: string
Filter (State#18 = LA)
+- Join Inner, (IATA#20 = origin#65)
   :- Relation [date#62,delay#63,distance#64,origin#65,destination#66] csv
   +- Relation [City#17,State#18,Country#19,IATA#20] csv

== Optimized Logical Plan ==
Join Inner, (IATA#20 = origin#65)
:- Filter isnotnull(origin#65)
:  +- InMemoryRelation [date#62, delay#63, distance#64, origin#65, destination#66], StorageLevel(disk, 1 replicas)
:        +- FileScan csv [date#62,delay#63,distance#64,origin#65,destination#66] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/datasets/departuredelays.csv], Parti

# "Солёный" JOIN

In [4]:
fruit_stats_columns = ["fruit", "items_count"]
fruit_stats_data = [("Apple", "20")] * 10 + [("Orange", "10"), ("Pineapple", "30")]

fruit_stats_data_rdd = spark.sparkContext.parallelize(fruit_stats_data)

fruit_stats_data_df = fruit_stats_data_rdd.toDF(fruit_stats_columns)

fruit_stats_data_df.show()

+---------+-----------+
|    fruit|items_count|
+---------+-----------+
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|    Apple|         20|
|   Orange|         10|
|Pineapple|         30|
+---------+-----------+



In [5]:
fruit_color_columns = ["fruit", "color"]
fruit_color_data = [("Apple", "red"), ("Orange", "orange"), ("Pineapple", "yellow")]

fruit_color_data_rdd = spark.sparkContext.parallelize(fruit_color_data)

fruit_color_data_df = fruit_color_data_rdd.toDF(fruit_color_columns)

fruit_color_data_df.show()

+---------+------+
|    fruit| color|
+---------+------+
|    Apple|   red|
|   Orange|orange|
|Pineapple|yellow|
+---------+------+



In [6]:
fruit_stats_data_df.groupBy(col('fruit').alias('partition')).agg(count(col('fruit')).alias('partition_size')).show()

+---------+--------------+
|partition|partition_size|
+---------+--------------+
|Pineapple|             1|
|   Orange|             1|
|    Apple|            10|
+---------+--------------+



In [7]:
salted_df = fruit_stats_data_df.withColumn('salt', (rand() * 3).cast('int'))
salted_df.show()

+---------+-----------+----+
|    fruit|items_count|salt|
+---------+-----------+----+
|    Apple|         20|   1|
|    Apple|         20|   0|
|    Apple|         20|   2|
|    Apple|         20|   2|
|    Apple|         20|   1|
|    Apple|         20|   0|
|    Apple|         20|   1|
|    Apple|         20|   2|
|    Apple|         20|   0|
|    Apple|         20|   1|
|   Orange|         10|   1|
|Pineapple|         30|   0|
+---------+-----------+----+



In [8]:
salted_df.groupBy((concat(col('fruit'), col('salt'))).alias('partition')) \
                            .agg(count(col('fruit')).alias('partition_size')).show()

+----------+--------------+
| partition|partition_size|
+----------+--------------+
|    Apple1|             4|
|   Orange1|             1|
|    Apple2|             3|
|    Apple0|             3|
|Pineapple0|             1|
+----------+--------------+



In [9]:
enriched_df = fruit_color_data_df \
    .select(col('fruit'), col('color'), explode(array([lit(i) for i in [0, 1, 2]])).alias('salt'))
enriched_df.show()

+---------+------+----+
|    fruit| color|salt|
+---------+------+----+
|    Apple|   red|   0|
|    Apple|   red|   1|
|    Apple|   red|   2|
|   Orange|orange|   0|
|   Orange|orange|   1|
|   Orange|orange|   2|
|Pineapple|yellow|   0|
|Pineapple|yellow|   1|
|Pineapple|yellow|   2|
+---------+------+----+



In [10]:
joined_df = salted_df.join(enriched_df, ['fruit', 'salt'])
joined_df.drop('salt').show()

+---------+-----------+------+
|    fruit|items_count| color|
+---------+-----------+------+
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|   Orange|         10|orange|
|Pineapple|         30|yellow|
+---------+-----------+------+



In [11]:
fruit_stats_data_df.join(fruit_color_data_df, 'fruit').show()

+---------+-----------+------+
|    fruit|items_count| color|
+---------+-----------+------+
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|    Apple|         20|   red|
|   Orange|         10|orange|
|Pineapple|         30|yellow|
+---------+-----------+------+



# Пример использования broadcast

In [12]:
airport_delays \
    .join(airport_codes, airport_codes['IATA'] == airport_delays['origin']) \
    .filter(col('State') == 'LA') \
    .show()

+-------+-----+--------+------+-----------+----------+-----+-------+----+
|   date|delay|distance|origin|destination|      City|State|Country|IATA|
+-------+-----+--------+------+-----------+----------+-----+-------+----+
|1011030|   63|     247|   AEX|        DFW|Alexandria|   LA|    USA| AEX|
|1011838|  -14|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1011204|   -8|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1011710|   18|     435|   AEX|        ATL|Alexandria|   LA|    USA| AEX|
|1011115|   -3|     435|   AEX|        ATL|Alexandria|   LA|    USA| AEX|
|1021030|   -5|     247|   AEX|        DFW|Alexandria|   LA|    USA| AEX|
|1020537|   -7|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1021851|   -3|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1021204|   21|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1020600|   -6|     435|   AEX|        ATL|Alexandria|   LA|    USA| AEX|
|1021710|  115|     435|   AEX|       

In [16]:
# С броадкастом

try: bc_airport_codes.unpersist()
except Exception: pass

bc_airport_codes = broadcast(airport_codes)

airport_delays \
    .join(bc_airport_codes, airport_codes['IATA'] == airport_delays['origin']) \
    .filter(col('State') == 'LA') \
    .show()

+-------+-----+--------+------+-----------+----------+-----+-------+----+
|   date|delay|distance|origin|destination|      City|State|Country|IATA|
+-------+-----+--------+------+-----------+----------+-----+-------+----+
|1011030|   63|     247|   AEX|        DFW|Alexandria|   LA|    USA| AEX|
|1011838|  -14|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1011204|   -8|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1011710|   18|     435|   AEX|        ATL|Alexandria|   LA|    USA| AEX|
|1011115|   -3|     435|   AEX|        ATL|Alexandria|   LA|    USA| AEX|
|1021030|   -5|     247|   AEX|        DFW|Alexandria|   LA|    USA| AEX|
|1020537|   -7|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1021851|   -3|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1021204|   21|     165|   AEX|        IAH|Alexandria|   LA|    USA| AEX|
|1020600|   -6|     435|   AEX|        ATL|Alexandria|   LA|    USA| AEX|
|1021710|  115|     435|   AEX|       

# Пример использования фильтра Блума

In [17]:
fire_incidents.select(col('Incident Number')).distinct().count()

31771

In [18]:
fire_calls.select(col('IncidentNumber')).distinct().count()

168571

In [19]:
fire_calls \
    .join(fire_incidents, fire_incidents['Incident Number'] == fire_calls['IncidentNumber']) \
    .select(col('IncidentNumber')).distinct().count()

3247

In [20]:
# Без фильтра

fire_calls \
    .join(fire_incidents, fire_incidents['Incident Number'] == fire_calls['IncidentNumber']) \
    .count()

3458

In [21]:
!pip install --no-cache-dir mmh3 bitarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 5.0 MB/s eta 0:00:00a 0:00:01


In [22]:
import math
import mmh3
from bitarray import bitarray
  
  
class BloomFilter(object):
  
    '''
    Class for Bloom filter, using murmur3 hash function
    '''
  
    def __init__(self, items_count, fp_prob):
        '''
        items_count : int
            Number of items expected to be stored in bloom filter
        fp_prob : float
            False Positive probability in decimal
        '''
        self.items_count = items_count
        
        # False possible probability in decimal
        self.fp_prob = fp_prob
  
        # Size of bit array to use
        self.size = self.get_size(items_count, fp_prob)
  
        # number of hash functions to use
        self.hash_count = self.get_hash_count(self.size, items_count)
  
        # Bit array of given size
        self.bit_array = bitarray(self.size)
  
        # initialize all bits as 0
        self.bit_array.setall(0)
  
    def add(self, item):
        '''
        Add an item in the filter
        '''
        digests = []
        for i in range(self.hash_count):
  
            # create digest for given item.
            # i work as seed to mmh3.hash() function
            # With different seed, digest created is different
            digest = mmh3.hash(item, i) % self.size
            digests.append(digest)
  
            # set the bit True in bit_array
            self.bit_array[digest] = True
        
    def union(self, other):
        """ Calculates the union of the two underlying bitarrays and returns
        a new bloom filter object."""
        new_bloom = self.copy()
        new_bloom.bit_array = new_bloom.bit_array | other.bit_array
        return new_bloom
  
    def check(self, item):
        '''
        Check for existence of an item in filter
        '''
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            if self.bit_array[digest] == False:
  
                # if any of bit is False then,its not present
                # in filter
                # else there is probability that it exist
                return False
        return True
    
    def copy(self):
        """Return a copy of this bloom filter.
        """
        new_filter = BloomFilter(self.items_count, self.fp_prob)
        new_filter.bit_array = self.bit_array.copy()
        return new_filter
    
    def set_bit_array(self, bit_array):
        self.bit_array = bit_array
  
    @classmethod
    def get_size(self, n, p):
        '''
        Return the size of bit array(m) to used using
        following formula
        m = -(n * lg(p)) / (lg(2)^2)
        n : int
            number of items expected to be stored in filter
        p : float
            False Positive probability in decimal
        '''
        m = -(n * math.log(p))/(math.log(2)**2)
        return int(m)
  
    @classmethod
    def get_hash_count(self, m, n):
        '''
        Return the hash function(k) to be used using
        following formula
        k = (m/n) * lg(2)
  
        m : int
            size of bit array
        n : int
            number of items expected to be stored in filter
        '''
        k = (m/n) * math.log(2)
        return int(k)

In [24]:
from random import shuffle
  
n = 20 #no of items to add
p = 0.05 #false positive probability
  
bloomf1 = BloomFilter(n,p)
bloomf2 = BloomFilter(n,p)
  
# words to be added
word_present1 = ['abound','abounds','abundance','abundant','accessible',
                'bloom','blossom','bolster','bonny','bonus','bonuses']
  
# words to be added
word_present2 = ['coherent','cohesive','colorful','comely','comfort',
                'gems','generosity','generous','generously','genial']
  
# word not added
word_absent = ['bluff','cheater','hate','war','humanity',
               'racism','hurt','nuke','gloomy','facebook',
               'geeksforgeeks','twitter']
  
for item in word_present1:
    bloomf1.add(item)
  
for item in word_present2:
    bloomf2.add(item)
    
bloomf = bloomf1.union(bloomf2)
  
shuffle(word_present1)
shuffle(word_present2)
shuffle(word_absent)
  
test_words = word_present1 + word_present2 + word_absent
shuffle(test_words)
for word in test_words:
    if bloomf.check(word):
        if word in word_absent:
            print("'{}' is a false positive!".format(word))
        else:
            print("'{}' is probably present!".format(word))
    else:
        print("'{}' is definitely not present!".format(word))

'comely' is probably present!
'bonuses' is probably present!
'cohesive' is probably present!
'war' is definitely not present!
'genial' is probably present!
'bloom' is probably present!
'nuke' is definitely not present!
'gems' is probably present!
'blossom' is probably present!
'colorful' is probably present!
'hate' is definitely not present!
'abounds' is probably present!
'generous' is probably present!
'bluff' is definitely not present!
'generously' is probably present!
'cheater' is definitely not present!
'abundance' is probably present!
'coherent' is probably present!
'generosity' is probably present!
'bonus' is probably present!
'racism' is definitely not present!
'humanity' is a false positive!
'accessible' is probably present!
'comfort' is probably present!
'hurt' is definitely not present!
'bolster' is probably present!
'twitter' is a false positive!
'gloomy' is definitely not present!
'abound' is probably present!
'facebook' is definitely not present!
'abundant' is probably pre

In [25]:
# Создаем фильтр

filterSize = 31771
prob = 0.05

def fill_bloom_filter(bf, items):
    for i in items:
        bf.add(str(i[0]))
    return bf

bloom_filter = BloomFilter(filterSize, prob)

general_bit_array = fire_incidents.select(col('Incident Number')).rdd \
    .mapPartitions(lambda p: [fill_bloom_filter(BloomFilter(filterSize, prob), p).bit_array]) \
    .reduce(lambda a, b: a.bit_array | b.bit_array)

bloom_filter.set_bit_array(general_bit_array)

maybe_in_bf = udf(lambda incident_number: bloom_filter.check(str(incident_number)))

In [26]:
# Используем фильтр

fire_calls \
    .filter(maybe_in_bf(col('IncidentNumber')) == True) \
    .join(fire_incidents, fire_incidents['Incident Number'] == fire_calls['IncidentNumber']) \
    .count()

3458

# Использование Bucketing

In [27]:
# Без бакетинга

fire_calls \
    .join(fire_incidents, fire_incidents['Incident Number'] == fire_calls['IncidentNumber']) \
    .count()

3458

In [29]:
fire_calls.write \
    .bucketBy(16, 'IncidentNumber') \
    .saveAsTable('fire_calls_bucketed', format='csv', mode='overwrite')
    
fire_incidents.write \
    .bucketBy(16, 'Incident Number') \
    .saveAsTable('fire_incidents_bucketed', format='csv', mode='overwrite')

In [30]:
# С бакетингом

fire_calls_bucketed = spark.table('fire_calls_bucketed')
fire_incidents_bucketed = spark.table('fire_incidents_bucketed')

fire_calls_bucketed \
    .join(fire_incidents_bucketed, fire_incidents_bucketed['Incident Number'] == fire_calls_bucketed['IncidentNumber']) \
    .count()

3458

In [31]:
str(1)

'1'

In [32]:
print(bc_airport_codes == None)

False


In [33]:
bc_airport_codes.unpersist()

DataFrame[City: string, State: string, Country: string, IATA: string]

In [34]:
airport_codes.select(col('State')).distinct().show()

+-------+
|  State|
+-------+
| Hawaii|
|     SC|
|     AZ|
|     NS|
|     LA|
|     MN|
|     NJ|
|    PEI|
|     OR|
|     VA|
|   null|
|     RI|
|     WY|
|     KY|
|     BC|
|     NH|
|     MI|
|     NV|
|    NWT|
|Nunavut|
+-------+
only showing top 20 rows



In [35]:
airport_delays.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [36]:
fire_incidents.select(col('Incident Number')).show()

+---------------+
|Incident Number|
+---------------+
|       16000003|
|       16000004|
|       16000023|
|       16000034|
|       16000051|
|       16000053|
|       16000064|
|       16000071|
|       16000072|
|       16000099|
|       16000143|
|       16000145|
|       16000155|
|       16000161|
|       16000167|
|       16000166|
|       16000171|
|       16000176|
|       16000179|
|       16000183|
+---------------+
only showing top 20 rows



In [37]:
fire_calls.select(col('IncidentNumber')).show()

+--------------+
|IncidentNumber|
+--------------+
|       2003235|
|       2003241|
|       2003242|
|       2003250|
|       2003259|
|       2003279|
|       2003301|
|       2003304|
|       2003343|
|       2003348|
|       2003381|
|       2003382|
|       2003399|
|       2003403|
|       2003408|
|       2003408|
|       2003408|
|       2003409|
|       2003417|
|       2003417|
+--------------+
only showing top 20 rows



In [38]:
fire_incidents \
    .join(fire_calls, fire_incidents['Incident Number'] == fire_calls['IncidentNumber']) \
    .show()

+---------------+---------------+--------------------+-------------+-----------+--------------------+--------------------+--------------------+-------------+-------+---------+------------+----+-----------------+---------------------+---------+-------------+-----------+---------------+-------------------+-----------------------+-----------------------+---------------+-------------+-------------------+-----------------+----------------+--------------------+----------+--------------------+----------------------+------------------+--------------------------+--------------------+--------------------+---------------+-----------------------+-------------------------+--------------------+--------------------+--------------------------------------+-------------------+----------------+--------------------+-----------+--------------+------------------------------------+----------------------------------------+----------------------------------+------------------------------------+----------------

In [39]:
airport_codes.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows

